In [7]:
import pandas as pd 
import json
import psycopg2
from config import config
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
#get data from db
def connect():
    params = config()
    
    print(**params)
    #conn = psycopg2.connect(**params)

connect()

NameError: name 'host' is not defined

In [ ]:
#This function sorts the cpu products by renaming and merging columns for consistency
def filter_cpu(data):
    data_des = data[['type', 'retailer', 'brand','# of CPU Cores', '# of Threads', 'Base Clock', 'Max Boost Clock',
                                        'Default TDP / TDP', 'Processor Base Frequency','Max Turbo Frequency', 'TDP']]
    
    df_change = data_des.loc[data_des['Base Clock'].isnull()]
    data_des.drop(df_change.index, inplace=True)
    df_change.drop(['Base Clock', 'Default TDP / TDP', 'Max Boost Clock'], axis=1, inplace=True)
    df_change.rename({'Processor Base Frequency': 'Base Clock', 'Max Turbo Frequency': 'Max Boost Clock'}, axis=1, inplace=True)
    
    data_des.drop(['Processor Base Frequency', 'Max Turbo Frequency', 'TDP'], axis=1, inplace=True)
    data_des.rename({'Default TDP / TDP': 'TDP'}, axis=1, inplace=True)
    data_des = pd.merge(data_des, df_change, how='outer')
    return data_des


In [ ]:
def main():
    #get users
    #get users wishlist; filter
    #get all products that are not in wishlist; filter
    #encode data 
    #predict